# Using Noise by Perturbing Each Image in a Certain Region

In [53]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class RegionalNoise():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()
        
        # Eliminated because it is just bad
        # shuffles 50000 training data
        #np.random.shuffle(x_train_bef)
        #np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype('float')
        x_test = x_test.astype('float')

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it takes 3 random 5x5 sections of each image and perturbs each section.
    # the input it original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert we look at a random location and perturb the 5x5 region
        length=x_train_pert.shape[0]
        for i in range(length):
            # perturbation is done 3 times
            for repeat in range(3):
                loc_x = random.randrange(2,30) # x-coord of center of 5x5 region
                loc_y = random.randrange(2,30) # y-coord of center of 5x5 region

                # each pixel and each RGB value is perturbed
                for u in [-2,-1,0,1,2]:
                    for v in [-2,-1,0,1,2]:
                        x_train_pert[i][loc_x+u][loc_y+v][0] += 50*(random.random()-0.5)
                        x_train_pert[i][loc_x+u][loc_y+v][1] += 50*(random.random()-0.5)
                        x_train_pert[i][loc_x+u][loc_y+v][2] += 50*(random.random()-0.5)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = RegionalNoise.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = RegionalNoise.generate_data(trainSize)
        if noisy:
            x_train, y_train = RegionalNoise.create_noise(x_train, y_train, Set_enlarge)
        MODEL = RegionalNoise.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize, validation_data=(x_test, y_test))
        MODEL.evaluate(x_test,y_test)

# Using Noise by Perturbing Each Pixel with Equal Probability (Salt and Pepper)

In [55]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class Salt_and_Pepper():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()
        
        # Eliminated because it is just bad
        # shuffles 50000 training data
        #np.random.shuffle(x_train_bef)
        #np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype('float')
        x_test = x_test.astype('float')

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it perturbs a pixel with probability 0.1 (and changes each RGB value by a random amount)
    # the input is original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert we perturb each pixel with probability 0.1
        shape=x_train_pert.shape
        for i in range(shape[0]):
            for x in range(shape[1]):
                for y in range(shape[2]):
                    num = random.random()
                    if num<0.1:
                        x_train_pert[i][x][y][0] += 100*(random.random()-0.5)
                        x_train_pert[i][x][y][1] += 100*(random.random()-0.5)
                        x_train_pert[i][x][y][2] += 100*(random.random()-0.5)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = Salt_and_Pepper.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = Salt_and_Pepper.generate_data(trainSize)
        if noisy:
            x_train, y_train = Salt_and_Pepper.create_noise(x_train, y_train, Set_enlarge)
        MODEL = Salt_and_Pepper.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize)
        MODEL.evaluate(x_test,y_test)

In [22]:
# Again, not much effect
Salt_and_Pepper.run_model(4,64,6000, noisy=True, Set_enlarge=10)

Epoch 1/4
938/938 [==============================] - 96s 99ms/step - loss: 4.3992 - accuracy: 0.1427
Epoch 2/4
938/938 [==============================] - 95s 101ms/step - loss: 2.1106 - accuracy: 0.5584
Epoch 3/4
938/938 [==============================] - 100s 107ms/step - loss: 1.5498 - accuracy: 0.8324
Epoch 4/4
1688/1688 [==============================] - 29s 17ms/step - loss: 7.8101 - accuracy: 0.0998


# Adding Gaussian Noise to Each Pixel

In [72]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
import random

class GaussianEvery():
    # returns the CIFAR-10 data set (info here: https://keras.io/api/datasets/cifar10/)
    # The data set loaded is has 50,000 training images and 10,000 testing images, but the function below only takes
    # a certain number train_size of training images from the original training set. The rest of the training examples
    # are added to the test images.
    def generate_data(train_size):
        # loads the data in 50000,10000 form with labels (y) as integers, not one-hot
        (x_train_bef, y_train_bef), (x_test_bef, y_test_bef) = tf.keras.datasets.cifar10.load_data()

        # shuffles 50000 training data
        #np.random.shuffle(x_train_bef)
        #np.random.shuffle(y_train_bef)

        # cuts off train_size amounts of sample data
        x_train = x_train_bef[:train_size]
        y_train = y_train_bef[:train_size]

        # adds the rest of the training data to the test data
        x_test = np.concatenate((x_train_bef[train_size:], x_test_bef), axis=0)
        y_test = np.concatenate((y_train_bef[train_size:], y_test_bef), axis=0)

        # turns image arrays into floats just so that everything is a float, not an int
        x_train = x_train.astype('float')
        x_test = x_test.astype('float')

        # turns label data into one-hot form
        y_train = tf.keras.utils.to_categorical(y_train)
        y_test = tf.keras.utils.to_categorical(y_test)

        return x_train, y_train, x_test, y_test

    # looks at the training data, clones it, but then creates small changes to each image
    # Currently, it adds a random amount (that varies according to Gaussian distribution) to each RGB value of each
    # pixel
    # the input is original training data which is not changed, but copied and then perturbed: the perturbed result
    # is returned
    def create_perturbed_clone(x_train, y_train):
        # new objects that are copied: changing x_train_pert won't change x_train
        x_train_pert = x_train.copy()
        y_train_pert = y_train.copy()

        # for each image in x_train_pert and each pixel of the image we perturb the RGB values by a Gaussian amount
        shape=x_train_pert.shape
        for i in range(shape[0]):
            for x in range(shape[1]):
                for y in range(shape[2]):
                    x_train_pert[i][x][y][0] += random.gauss(0,20)
                    x_train_pert[i][x][y][1] += random.gauss(0,20)
                    x_train_pert[i][x][y][2] += random.gauss(0,20)

        return x_train_pert, y_train_pert

    # takes training data and adds a perturbed copy to make the training set enlarge_factor times larger
    def create_noise(x_train, y_train, enlarge_factor):
        x_train_noisy = x_train.copy()
        y_train_noisy = y_train.copy()
        for i in range(enlarge_factor-1):
            x_add, y_add = GaussianEvery.create_perturbed_clone(x_train, y_train)
            x_train_noisy = np.concatenate((x_train_noisy, x_add))
            y_train_noisy = np.concatenate((y_train_noisy, y_add))
        return x_train_noisy, y_train_noisy

    # creates network: we use a convolutional neural network which makes sense for this problem
    def create_model():
        model = Sequential()
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used

        
        #opt = keras.optimizers.SGD(learning_rate = 0.1, momentum = 0.9)
        #model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        return model

    def run_model(Epochs, BatchSize, trainSize, Set_enlarge, noisy=False):
        x_train, y_train, x_test, y_test = GaussianEvery.generate_data(trainSize)
        if noisy:
            x_train, y_train = GaussianEvery.create_noise(x_train, y_train, Set_enlarge)
        MODEL = GaussianEvery.create_model()
        MODEL.fit(x=x_train, y=y_train, epochs=Epochs, batch_size=BatchSize)
        MODEL.evaluate(x_test,y_test)

In [73]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 18s 98ms/step - loss: 25.8135 - accuracy: 0.3554
Epoch 2/5
141/141 [==============================] - 9s 62ms/step - loss: 1.9253 - accuracy: 0.9787
Epoch 3/5
141/141 [==============================] - 9s 67ms/step - loss: 1.6000 - accuracy: 0.9831
Epoch 4/5
141/141 [==============================] - 10s 71ms/step - loss: 1.4212 - accuracy: 0.9931
Epoch 5/5
1866/1866 [==============================] - 39s 21ms/step - loss: 8.8236 - accuracy: 0.2663


In [74]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 10s 64ms/step - loss: 15.4400 - accuracy: 0.2788
Epoch 2/5
141/141 [==============================] - 9s 66ms/step - loss: 1.5555 - accuracy: 0.9817
Epoch 3/5
141/141 [==============================] - 9s 67ms/step - loss: 1.1894 - accuracy: 0.9934
Epoch 4/5
141/141 [==============================] - 10s 68ms/step - loss: 1.2052 - accuracy: 0.9685
Epoch 5/5
1866/1866 [==============================] - 38s 20ms/step - loss: 6.6954 - accuracy: 0.2452


In [75]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 12s 78ms/step - loss: 17.5802 - accuracy: 0.3886
Epoch 2/5
141/141 [==============================] - 12s 82ms/step - loss: 1.8126 - accuracy: 0.9944
Epoch 3/5
141/141 [==============================] - 11s 76ms/step - loss: 1.6058 - accuracy: 0.9547
Epoch 4/5
141/141 [==============================] - 12s 82ms/step - loss: 1.1785 - accuracy: 1.0000
Epoch 5/5
1866/1866 [==============================] - 58s 31ms/step - loss: 6.3782 - accuracy: 0.2794


In [76]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 16s 108ms/step - loss: 23.6108 - accuracy: 0.3460
Epoch 2/5
141/141 [==============================] - 16s 114ms/step - loss: 1.8023 - accuracy: 0.9776
Epoch 3/5
141/141 [==============================] - 17s 118ms/step - loss: 1.4260 - accuracy: 0.9886
Epoch 4/5
141/141 [==============================] - 19s 138ms/step - loss: 1.2121 - accuracy: 0.9955
Epoch 5/5
1866/1866 [==============================] - 40s 21ms/step - loss: 7.6419 - accuracy: 0.2628


In [77]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 10s 67ms/step - loss: 22.9743 - accuracy: 0.3046
Epoch 2/5
141/141 [==============================] - 13s 94ms/step - loss: 1.7363 - accuracy: 0.9772
Epoch 3/5
141/141 [==============================] - 13s 95ms/step - loss: 1.3452 - accuracy: 0.9848
Epoch 4/5
141/141 [==============================] - 13s 95ms/step - loss: 1.2807 - accuracy: 0.9798
Epoch 5/5
1866/1866 [==============================] - 65s 35ms/step - loss: 8.1792 - accuracy: 0.26290s - loss: 8.1789 - accuracy: 0.


In [78]:
GaussianEvery.run_model(5, 32, 300, 15, noisy=True)

Epoch 1/5
141/141 [==============================] - 23s 157ms/step - loss: 21.3188 - accuracy: 0.2970
Epoch 2/5
141/141 [==============================] - 23s 164ms/step - loss: 1.8368 - accuracy: 0.9775
Epoch 3/5
141/141 [==============================] - 26s 188ms/step - loss: 1.4676 - accuracy: 0.9900
Epoch 4/5
141/141 [==============================] - 27s 189ms/step - loss: 1.2572 - accuracy: 0.9873
Epoch 5/5
1866/1866 [==============================] - 84s 45ms/step - loss: 8.2008 - accuracy: 0.2209


In [79]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 24s 161ms/step - loss: 27.2569 - accuracy: 0.3024
Epoch 2/2
1866/1866 [==============================] - 84s 45ms/step - loss: 7.2915 - accuracy: 0.2577


In [80]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 30s 198ms/step - loss: 20.9560 - accuracy: 0.2550
Epoch 2/2
1866/1866 [==============================] - 123s 66ms/step - loss: 6.4821 - accuracy: 0.2237s - loss: 6.4850  - ETA: 0s - loss: 6.4831 - 


In [81]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 31s 210ms/step - loss: 15.2398 - accuracy: 0.3938
Epoch 2/2
1866/1866 [==============================] - 104s 56ms/step - loss: 7.4016 - accuracy: 0.2365


In [82]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 28s 189ms/step - loss: 15.5353 - accuracy: 0.3542
Epoch 2/2
1866/1866 [==============================] - 92s 49ms/step - loss: 8.0374 - accuracy: 0.2523


In [83]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 25s 168ms/step - loss: 23.2110 - accuracy: 0.3307
Epoch 2/2
1866/1866 [==============================] - 87s 46ms/step - loss: 7.1248 - accuracy: 0.2586


In [84]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 23s 156ms/step - loss: 15.8861 - accuracy: 0.3053
Epoch 2/2
1866/1866 [==============================] - 129s 69ms/step - loss: 7.3667 - accuracy: 0.2463


In [85]:
GaussianEvery.run_model(2, 32, 300, 15, noisy=True)

Epoch 1/2
141/141 [==============================] - 28s 188ms/step - loss: 17.5231 - accuracy: 0.3743
Epoch 2/2
1866/1866 [==============================] - 91s 49ms/step - loss: 6.5445 - accuracy: 0.2358


# Summary

300 training samples: batchsize 32

Without noise: (50 epochs)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.3197|0.2531|0.9314|1.0000
2|7.5642|0.2684|0.9227|1.0000
3|7.2833|0.2283|1.4095|0.9393
4|8.5081|0.2570|0.8407|1.0000
5|6.9334|0.2533|1.0045|1.0000

Without noise: (200 epochs)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.5630|0.2409|0.1992|1.0000
2|8.4634|0.2221|0.2613|1.0000
3|7.7740|0.2253|0.1663|1.0000

Without noise: (30 epochs)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.6429|0.2759|1.4202|1.0000
2|7.2435|0.2595|1.0762|1.0000
3|7.3298|0.2426|1.3551|0.9901
4|8.3097|0.2665|1.4177|1.0000
5|8.0503|0.2644|1.2590|1.0000
6|8.5601|0.2667|1.3722|1.0000
7|7.4139|0.2569|1.3948|0.9956
Mean|7.7929|0.2618|1.3279|0.9980

Without noise: (75 epochs)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|6.5455|0.2532|0.7241|1.0000
2|8.6308|0.2390|0.5513|1.0000
3|8.8108|0.2274|0.6607|1.0000
4|6.7910|0.2751|0.7818|1.0000
5|8.4803|0.2537|0.8462|1.0000
6|6.1162|0.2283|1.2994|0.9643
7|7.6539|0.2395|0.7188|1.0000
Mean|7.5755|0.2452|0.7975|0.9949

Without noise: (300 epochs)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|5.4548|0.1971|1.7993|0.7224
2|9.0567|0.2033|0.1067|1.0000
3|9.8404|0.2128|0.1902|1.0000

With regional noise: (2 epochs, x15 enlargement) -25,25 pert in 3 5x5's

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.4306|0.2700|1.4275|0.9873
2|8.8499|0.2348|1.9837|0.9888
3|7.1080|0.2674|1.7379|0.9848
4|7.0256|0.2651|1.5353|0.9888
5|7.0098|0.3022|1.7845|0.9910
6|6.0001|0.3022|1.3452|1.0000
7|9.5572|0.2377|1.9317|0.9916
Mean|7.5687|0.2685|1.6780|0.9903

With regional noise: (5 epochs, x15 enlargement) -25,25 pert in 3 5x5's

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|5.5464|0.2939|0.8340|1.0000
2|6.8525|0.2761|0.7920|1.0000
3|6.1521|0.2790|0.7198|1.0000
4|6.3310|0.2971|0.8985|1.0000
5|5.9695|0.2671|0.7849|1.0000
6|5.3779|0.3119|0.6422|1.0000
Mean|6.0382|0.2875|0.7786|1.0000

With regional noise: (20 epochs, x15 enlargement) -25,25 pert in 3 5x5's

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.4443|0.2350|0.2884|1.0000
2|8.7246|0.2342|1.2210|0.8859
3|6.3376|0.2561|0.2901|1.0000

With salt and pepper noise: (2 epochs, x15 enlargement) -50,50 pert for each pixel with 0.1 probability

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.7568|0.2455|1.4054|0.9950
2|7.9804|0.2577|1.6313|0.9904
3|8.4064|0.2453|1.7768|0.9898
4|8.4090|0.2261|1.7916|0.9204
5|7.3795|0.2772|1.7459|0.9961
6|6.6957|0.2587|1.6473|0.9897
7|8.5838|0.2774|1.8218|0.9525
Mean|0.7887|0.2554|1.6886|0.9763

With salt and pepper noise: (5 epochs, x15 enlargement) -50,50 pert for each pixel with 0.1 probability

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.1246|0.2537|0.7972|1.0000
2|8.0320|0.2594|1.0688|0.9940
3|8.4289|0.2401|1.1869|1.0000
4|7.2578|0.2641|1.0126|1.0000
5|5.5459|0.2262|0.8560|0.9853
6|7.2431|0.2557|0.9617|1.0000
Mean|7.2721|0.2499|0.9805|0.9966

With Gaussian noise at every pixel: (2 epochs, x15 enlargement) mean=0, std=20 perturbation with each pixel

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|6.5445|0.2358|1.7699|0.9937
2|7.3667|0.2463|1.9940|0.9623
3|7.1248|0.2586|1.6657|0.9876
4|8.0374|0.2523|1.5926|0.9814
5|7.4016|0.2365|1.8036|0.9753
6|6.4821|0.2237|1.8749|0.9504
7|7.2915|0.2577|1.7168|0.9518
Mean|7.1784|0.2444|1.7739|0.9718

With Gaussian noise at every pixel: (5 epochs, x15 enlargement) mean=0, std=20 perturbation with each pixel

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|8.2008|0.2209|1.1386|0.9855
2|8.1792|0.2629|0.9754|1.0000
3|7.6419|0.2628|0.9691|1.0000
4|6.3782|0.2794|0.9536|1.0000
5|6.6954|0.2452|0.8889|0.9986
6|8.8236|0.2663|1.3215|0.9864
Mean|7.6532|0.2563|1.0412|0.9951

Note: Everything below is bad because the code messed up the training data.

The neural net is unable to generalize off of 6,000 training samples (54,000 testing) in general. This is also the case when we add the three types of noise from above.

We now investigate if we increase trainin size and see what noise does. We change the architecture of the network a little to account for this (still using relu activation and a cnn, however).

Architecture:

        model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        #model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_regularizer='l2'))
        model.add(Dense(30, activation='relu'))
        model.add(Dense(10, activation='softmax')) # output lyer is 10-dimension one-hot, so softmax is used


15,000 train, 45,000 test: 4 epochs, batch size 128, noise multiplies training set by 10
    
Without Noise: (12 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|3.1619|0.1016|2.4222|0.3392
2|2.8962|0.0966|2.4348|0.2762
3|3.4867|0.0999|2.3478|0.3993
4|2.8908|0.1022|2.4825|0.2516
5|2.9787|0.0966|2.3811|0.3023

Without Noise: (40 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|7.1343|0.0982|1.7263|0.8225
2|5.6680|0.1012|2.1604|0.6673

With Regional Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|5.0367|0.0994|2.5729|0.4287
2|4.9546|0.0970|2.4516|0.4538

With Salt and Pepper Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|4.1155|0.1032|2.7049|0.2644
2|4.8277|0.0986|2.6190|0.3976

With Every Pixel Gaussian Noise: (2 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|4.1773|0.0966|2.7137|0.2648
2|4.0520|0.1019|2.7116|0.2949

With Regional Noise: (5 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|6.9880|0.1032|1.7123|0.8542
2|7.2141|0.1015|1.7234|0.8455
1|6.8830|0.1006|1.8362|0.8523

With Salt and Pepper Noise: (5 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|6.6406|0.1004|1.8135|0.8009
2|6.7533|0.0999|1.9012|0.8332
3|6.6961|0.0992|1.8420|0.8268

With Every Pixel Gaussian Noise: (5 epochs used)

Trial |validation loss |validation accuracy |training loss |training accuracy
-|-|-|-|-
1|5.6420|0.0980|2.0358|0.6315
2|5.7523|0.0992|2.1808|0.7011
3|5.8519|0.1001|2.0394|0.7118

Normal 50,000 training:

Architecture:

        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(10, activation='softmax'))
        
losses: 2.35, 2.4 (training, val); 
accuracy: 0.146, 0.1139

Similar to previous architecture's results 

This is likely significant (i.e. our net does stuff with lots of training)